In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('smsspamcollection.tsv',sep='\t')

In [4]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [5]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [7]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
##using text as a feature
X = df['message']

In [10]:
y = df['label']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
count_vect = CountVectorizer()

In [14]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [16]:
#X is still in this form, we have to vectorize it
#way 1
#first vectorizer to data, (build a vocab, count no of words)
#trnasform original mssg to vector

In [17]:
#count_vect.fir(X_train)  #build vocab
#X_train_counts = count_vect.transform(X_train) #transform

#or easy, combining above two commands
X_train_counts = count_vect.fit_transform(X_train)

#X-train_counts is a sparce matrix

In [18]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [19]:
X_train.shape

(3733,)

In [20]:
###across mssgs, 7082 unique words


In [22]:
X_train_counts.shape

(3733, 7082)

In [23]:
#TF_IDF
# trem freq inverse doc freq
from sklearn.feature_extraction.text import TfidfTransformer

In [25]:
#create instance
tfidf_transformer = TfidfTransformer()

In [27]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [29]:
X_train_tfidf.shape

(3733, 7082)

In [34]:
# combining above steps of count vectorization and tfidf transformation
# alternative:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [36]:
from sklearn.svm import LinearSVC

In [37]:
clf = LinearSVC()

In [39]:
clf.fit(X_train_tfidf,y_train)  ##will display model used

LinearSVC()

In [ ]:
#pipeline class can be used to vectorize and classification
# everything above in:

from sklearn.pipeline import Pipeline

text_clf = Pipeline([('')])